In [ ]:
# Attempt on hashing using SHA-256 (NOT part of the Cheon Algorithm)
# ---

import hashlib, binascii
from sage.crypto.util import ascii_to_bin

# Pseudorandom function
def f_s(ec_point, s):
    x, y = ec_point.xy()

    hx = hashlib.sha3_256(ZZ(x).binary().encode('utf-16')).digest()
    hy = hashlib.sha3_256(ZZ(y).binary().encode('utf-16')).digest()

    x_hat = int.from_bytes(binascii.hexlify(hx), byteorder='big')
    y_hat = int.from_bytes(binascii.hexlify(hy), byteorder='big')
                          
    return (x_hat + y_hat) % s

# Example usage
point_to_hash = E(920, 303)
hashed_point = f_s(point_to_hash, 70)

print(f"Original EC Point: {point_to_hash}")
print(f"Hashed EC Point: {hashed_point}")


In [1]:
def prf(G, s):
    if G == E(0,1,0): 
        return 1
    return (int(G.xy()[0]) + int(G.xy()[1])) % s + 1

# Reference: EC Handbook (Sec 19.4)
def bsgs(zeta, s, left_G, right_G, start_G):
    hash_table = dict()
    for j in range(s): # construct hash table {hg^{-1}, hg^{-2}, ..., hg^{-s+1}} where 's' is a divisor of EC order
        print(right_G + zeta^(-j) * left_G)
        hash_table[right_G + zeta^(-j) * left_G] = j

    gamma = start_G
    for i in range(s):
        if (hash_table.get(gamma)):
            print("BSGS Collision")
            j = hash_table[gamma]
            return j + i * s
        gamma = gamma + s * G
        
    return 0

def pollard_kangaroo(left_G, right_G, zeta, s, start_G, c1, c2):                                                             
    hash_table = dict()

    tame_dist, tame_walk = 0, zeta^c1 * left_G
    for j in range(s): # construct hash table for a random set of points
        hash_table[tame_walk] = j
        tame_dist += prf(F(P, s, zeta, j), s)
        tame_walk = zeta^tame_dist * P
    
    wild_dist, wild_walk = 0, zeta^c2 * right_G
    for i in range(s):
        wild_dist += prf(F(Q, s, zeta, i), s)
        wild_walk = zeta^wild_dist * right_G
        if wild_walk == tame_walk:
            print("Kangaroo Collision")
            return (tame_dist - wild_dist) % s
    
    return 0

In [6]:
import random

# Cheon Algorithms to solve DLPwAI
# ---

alpha = 3347 # private key 

# toy example from Hoffstein Textbook (Sec 6.4)
p = 3851
K = GF(p)
a = 324
b = 1287
E = EllipticCurve(K, [a,b])
n = E.order() # a prime number
m = len((n-1).divisors())

# choose r to be largest prime number that divides EC order
for i in range(m-1,1,-1):
    if ((n-1).divisors()[i].is_prime()):
        r = (n-1).divisors()[i]
        break

d = (r-1).divisors()[1] # How is this divisor chosen such that runtime is optimal?
d_prime = (r-1)/d

# Input
G = E(920, 303)
G_1 = alpha * G
G_d = alpha^d * G

for i in range(1,r):
    if (mod(i, r).is_primitive_root()):
        zeta = i
        break

        
# Cheon BSGS

start_timestamp = current_milli_time()

k_1 = bsgs(zeta^d, ceil(sqrt(d_prime)), G_d, G, E(0,1,0)) # appropriate search interval [a,b]?
k_2 = bsgs(zeta^d_prime, ceil(sqrt(d)), G_1, G, zeta^k_1 * G)
alpha = zeta ^ (k_1 + k_2*d)
print("Found secret key %d using BSGS", alpha)

end_timestamp = current_milli_time()
print('Runtime (in ms):', end_timestamp - start_timestamp)

# Cheon Kangaroo

start_timestamp = current_milli_time()

k_1 = pollard_kangaroo(G_d, G, zeta^d, ceil(sqrt(d_prime)), 2, 3)
k_2 = pollard_kangaroo(G_1, zeta^k_1 * G, zeta^d_prime, ceil(sqrt(d)), 2, 3)
alpha = zeta ^ (k_1 + k_2*d_prime)
print("Found secret key %d using Kangaroo", alpha)

end_timestamp = current_milli_time()
print('Runtime (in ms):', end_timestamp - start_timestamp)

(1565 : 1584 : 1)


TypeError: unsupported operand type(s) for *: 'float' and 'EllipticCurvePoint_finite_field'